<a href="https://colab.research.google.com/github/Alex-Witt/DS-Unit-1-Sprint-2-Data-Wrangling/blob/master/module2-join-datasets/LS_DS_122_Join_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Lambda School Data Science_

# Join datasets

Objectives
- concatenate data with pandas
- merge data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
  - Combine Data Sets: Standard Joins
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join

## Download data

We’ll work with a dataset of [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)!

In [1]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-03-26 22:13:08--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.238.173
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.238.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.3’

instacart_online_gr 100%[===================>] 196.03M  25.5MB/s    in 14s     

2019-03-26 22:13:22 (14.4 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.3’ saved [205548478/205548478]



In [2]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [3]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


## Goal: Reproduce this example

The first two orders for user id 1:

In [4]:
from IPython.display import display, Image
url = 'https://cdn-images-1.medium.com/max/1600/1*vYGFQCafJtGBBX5mbl0xyw.png'
example = Image(url=url, width=600)

display(example)

## Load data

Here's a list of all six CSV filenames

In [5]:
!ls -lh

total 681M
-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


For each CSV
- Load it with pandas
- Look at the dataframe's shape
- Look at its head (first rows)
- `display(example)`
- Which columns does it have in common with the example we want to reproduce?

### aisles

In [6]:
import pandas as pd
aisles = pd.read_csv('aisles.csv')
print(aisles.shape)
aisles.head()

(134, 2)


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [7]:
display(example)

aisles doesnt have any columns we need. 

### departments

In [8]:
departments = pd.read_csv('departments.csv')
print(departments.shape)
departments.head()

(21, 2)


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


departments does not have any colum we need

### order_products__prior

In [9]:
order_products_prior = pd.read_csv('order_products__prior.csv')
print(order_products_prior.shape)
order_products_prior.head()

(32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [10]:
order_products_prior.groupby('order_id')['product_id'].count().mean()

10.088883421247614

### order_products_prior has stuff we need
- order_id
- product_id
- add_to_cart_order

### order_products__train

In [11]:
order_products_train = pd.read_csv('order_products__train.csv')
print(order_products_train.shape)
order_products_train.head()

(1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


**order_products_prior** and **order_products_train** have the exact same columns

### orders

In [12]:
orders = pd.read_csv('orders.csv')
print(orders.shape)
orders.head()

(3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


**orders** has:

- user_id
- order_id
- order_numbers
- order_dow
- order_hour_of_day

### products

In [13]:
products = pd.read_csv('products.csv')
print(products.shape)
products.head()

(49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


**products** has

- product_id
- product name

## Concatenate order_products__prior and order_products__train

In [0]:
order_products = pd.concat([order_products_prior, order_products_train])

In [15]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [16]:
order_products.shape, order_products_prior.shape, order_products_train.shape

((33819106, 4), (32434489, 4), (1384617, 4))

In [17]:
len(order_products) == len(order_products_prior) + len(order_products_train)

True

In [18]:
(len(order_products.columns) == 
 len(order_products_prior.columns) == 
 len(order_products_train.columns))

True

In [19]:
rows, columns = order_products.shape

print(rows)
print(columns)

33819106
4


## Get a subset of orders — the first two orders for user id 1

From `orders` dataframe
- user_id
- order_id
- order_number
- order_dow
- order_hour_of_day


In [0]:
condition = (orders['user_id']==1) & (orders['order_number']<=2)    #[:2] or other slice notation

columns = ['user_id','order_id','order_number','order_dow','order_hour_of_day']

subset = orders.loc[condition, columns]

In [21]:
subset

,user_id,order_id,order_number,order_dow,order_hour_of_day
0,1,2539329,1,2,8
1,1,2398795,2,3,7


In [0]:
###Text for notes purposes. 

#order_products[order_id[]]
#condition = order_products['order_id'] == 2539329
#order_products[condition]

## Merge dataframes

Merge the subset from orders with columns from order_products

In [0]:
merged = pd.merge(subset, 
         order_products[['order_id','add_to_cart_order','product_id']],
                 how = 'inner', on ='order_id')

In [24]:
subset.shape, order_products.shape, merged.shape

((2, 5), (33819106, 4), (11, 7))

In [25]:
display(example)

In [26]:
final = pd.merge(merged, products[['product_id','product_name']])
merged.shape, products[[ 'product_id', 'product_name']].shape, final.shape

((11, 7), (49688, 2), (11, 8))

In [27]:
final.sort_values(by =['order_number','add_to_cart_order'])

final

,user_id,order_id,order_number,order_dow,order_hour_of_day,add_to_cart_order,product_id,product_name
0,1,2539329,1,2,8,1,196,Soda
1,1,2398795,2,3,7,1,196,Soda
2,1,2539329,1,2,8,2,14084,Organic Unsweetened Vanilla Almond Milk
3,1,2539329,1,2,8,3,12427,Original Beef Jerky
4,1,2398795,2,3,7,3,12427,Original Beef Jerky
5,1,2539329,1,2,8,4,26088,Aged White Cheddar Popcorn
6,1,2398795,2,3,7,5,26088,Aged White Cheddar Popcorn
7,1,2539329,1,2,8,5,26405,XL Pick-A-Size Paper Towel Rolls
8,1,2398795,2,3,7,2,10258,Pistachios
9,1,2398795,2,3,7,4,13176,Bag of Organic Bananas


# Assignment

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.



In [28]:
sold_counts =(order_products['product_id']
              .value_counts().head(10))

sold_counts = pd.DataFrame(sold_counts)

sold_counts = sold_counts.reset_index()

sold_counts

,index,product_id
0,24852,491291
1,13176,394930
2,21137,275577
3,21903,251705
4,47209,220877
5,47766,184224
6,47626,160792
7,16797,149445
8,26209,146660
9,27845,142813


In [29]:
sold_counts = sold_counts.rename(columns = {'index':'product_id', 'product_id':'purchase_count'})
print(type(sold_counts))
print(sold_counts.shape)
print()
print(products.loc[products['product_id'] == 24852])
print()
sold_counts

<class 'pandas.core.frame.DataFrame'>
(10, 2)

       product_id product_name  aisle_id  department_id
24851       24852       Banana        24              4



,product_id,purchase_count
0,24852,491291
1,13176,394930
2,21137,275577
3,21903,251705
4,47209,220877
5,47766,184224
6,47626,160792
7,16797,149445
8,26209,146660
9,27845,142813


In [0]:
best_sellers = pd.merge(sold_counts,
        products[['product_id','product_name']],
        on = 'product_id').head(10)
         #how = 'right', on = product_list['product_id'])

In [31]:
best_sellers

,product_id,purchase_count,product_name
0,24852,491291,Banana
1,13176,394930,Bag of Organic Bananas
2,21137,275577,Organic Strawberries
3,21903,251705,Organic Baby Spinach
4,47209,220877,Organic Hass Avocado
5,47766,184224,Organic Avocado
6,47626,160792,Large Lemon
7,16797,149445,Strawberries
8,26209,146660,Limes
9,27845,142813,Organic Whole Milk


# Stretch challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.

In [32]:
late_orders = orders[orders['order_hour_of_day'] >= 12]
late_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
2,473747,1,prior,3,3,12,21.0
4,431534,1,prior,5,4,15,28.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
17,3194192,2,prior,7,2,12,14.0


In [33]:

sold_counts =(order_products['product_id']
              .value_counts())

sold_counts = pd.DataFrame(data = {'product_id': sold_counts.index, 'purchase_counts': sold_counts.values})

sold_counts = sold_counts[sold_counts['purchase_counts'] >= 2900]

sold_counts.head()

#sold_counts = sold_counts.reset_index()

#sold_counts = sold_counts.rename(columns = {'index':'product_id', 'product_id':'purchase_count'})



,product_id,purchase_counts
0,24852,491291
1,13176,394930
2,21137,275577
3,21903,251705
4,47209,220877


In [34]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [0]:
ids = pd.merge(order_products[['order_id','product_id']],
        late_orders[['order_id','order_hour_of_day']],
        on = 'order_id')

In [37]:
pd.merge(ids[['order_id','product_id']],
        sold_counts, on = 'product_id')

,order_id,product_id,purchase_counts
0,3,33754,30866
1,342,33754,30866
2,346,33754,30866
3,582,33754,30866
4,1171,33754,30866
5,1502,33754,30866
6,1662,33754,30866
7,1951,33754,30866
8,2166,33754,30866
9,2325,33754,30866
